# 2 Using Command Line Tools

Objectives:
 * Learn about the command line tools that are included with the HDF5 library (and h5pyd package for HSDS)
 * You can run these in a notebook using the shell escape '!', or open a terminal window and run them from there
 

In [1]:
# get a sample HDF5 file
! wget https://s3.amazonaws.com/hdfgroup/data/hdf5test/tall.h5

[0] Downloading 'https://s3.amazonaws.com/hdfgroup/data/hdf5test/tall.h5' ...
Saving 'tall.h5.2'
HTTP response 200 OK [https://s3.amazonaws.com/hdfgroup/data/hdf5test/tall.h5]


In [2]:
# Display objects with a file. Use -r for recursive mode
! h5ls -r tall.h5

/                        Group
/g1                      Group
/g1/g1.1                 Group
/g1/g1.1/dset1.1.1       Dataset {10, 10}
/g1/g1.1/dset1.1.2       Dataset {20}
/g1/g1.2                 Group
/g1/g1.2/extlink         External Link {somefile//somepath}
/g1/g1.2/g1.2.1          Group
/g1/g1.2/g1.2.1/slink    Soft Link {somevalue}
/g2                      Group
/g2/dset2.1              Dataset {10}
/g2/dset2.2              Dataset {3, 5}


In [3]:
# h5dump will show the dataset contents by default
! h5dump -r tall.h5

HDF5 "tall.h5" {
GROUP "/" {
   ATTRIBUTE "attr1" {
      DATATYPE  H5T_STD_I8BE
      DATASPACE  SIMPLE { ( 10 ) / ( 10 ) }
      DATA {
         "abcdefghi\000"
      }
   }
   ATTRIBUTE "attr2" {
      DATATYPE  H5T_STD_I32BE
      DATASPACE  SIMPLE { ( 2, 2 ) / ( 2, 2 ) }
      DATA {
      (0,0): 0, 1,
      (1,0): 2, 3
      }
   }
   GROUP "g1" {
      GROUP "g1.1" {
         DATASET "dset1.1.1" {
            DATATYPE  H5T_STD_I32BE
            DATASPACE  SIMPLE { ( 10, 10 ) / ( 10, 10 ) }
            DATA {
            (0,0): 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            (1,0): 0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
            (2,0): 0, 2, 4, 6, 8, 10, 12, 14, 16, 18,
            (3,0): 0, 3, 6, 9, 12, 15, 18, 21, 24, 27,
            (4,0): 0, 4, 8, 12, 16, 20, 24, 28, 32, 36,
            (5,0): 0, 5, 10, 15, 20, 25, 30, 35, 40, 45,
            (6,0): 0, 6, 12, 18, 24, 30, 36, 42, 48, 54,
            (7,0): 0, 7, 14, 21, 28, 35, 42, 49, 56, 63,
            (8,0): 0, 8, 16, 24, 32, 40, 48, 

In [4]:
# On the HSDS side, hsinfo will show the current server status
! hsinfo

server name: Highly Scalable Data Service (HSDS)
server state: READY
endpoint: http://localhost:5101
username: john (admin)
password: ****
home: NO ACCESS
server version: 0.8.3
node count: 4
up: 40 min 55 sec
h5pyd version: 0.15.1


In [5]:
# you can use hsload to copy a local file to HSDS
! hsload tall.h5 /

In [6]:
# hsls works like h5ls but with content managed by the server
! hsls -r /tall.h5

/ Group
/g1 Group
/g1/g1.1 Group
/g1/g1.1/dset1.1.1 Dataset {10, 10}
/g1/g1.1/dset1.1.2 Dataset {20}
/g1/g1.2 Group
/g1/g1.2/g1.2.1 Group
/g1/g1.2/g1.2.1/slink    SoftLink {somevalue}
/g1/g1.2/extlink         ExternalLink {somepath//somefile}
/g2 Group
/g2/dset2.1 Dataset {10}
/g2/dset2.2 Dataset {3, 5}


In [7]:
# Use the hstouch can be used to create a new folder (note: trailing slash is needed)
# let's use hscp to copy the tall.h5 in the root folder to our new folder and
# then list the contents of the folder
! hstouch /tutorial/
! hscp /tall.h5 /tutorial/
! hsls /tutorial/

Can not update timestamp of folder object
load_file, compressLevel: None
john                                                folder   2023-09-21 10:21:33 /tutorial/
john                                                domain   2023-09-21 10:39:58 /tutorial/tall.h5
2 items


Problem: Try the above hsls command with the -H -v flags

Problem: Try the above command with different options:
* -v
* --showattrs
* --showacls
* --loglevel debug


### Download the file 
Run: `$ hsget /tutorial/tall.h5 tall2.h5`
This will download the file as tall2.h5


### Compare the two files
Run: `$ h5dump tall.h5` 
and `$h5dump tall2.h5`

Problem: Are these files the same?

In [11]:
### hsstat
# hsstat provides information about storage used for HSDS objects similar to how h5stat works
# Note: it may take a minute before the information displayed by hsstat is current


In [12]:
# If no information is returned, wait a minute and try again!
! hsstat /tutorial/tall.h5

domain: /tutorial/tall.h5
    owner:           john
    id:              g-ad1a65c6-2c6d99f5-2592-ed4ea4-e5117d
    last modified:   2023-09-21 10:27:42
    last scan:       2023-09-21 10:28:50
    md5 sum:         813df5e3ca847eb66ed783e2adcdd45d
    total_size:      6017
    allocated_bytes: 580
    metadata_bytes:  5185
    num objects:     10
    num chunks:      4


### ACLS (Access Control List)
Each server domain or folder can contain one or more ACLs that control
who may perform operations (e.g. read/update/delete)




In [8]:
# Run hsacl
! hsacl /tutorial/tall.h5

       username     create      read    update    delete   readACL  updateACL 
--------------------------------------------------------------------------------
           john       True      True      True      True      True      True 


Problem: add an acl that gives all users read permissions (see hsacl --help)